# RNA-Seq Analysis using Nextflow and AWS Batch

## Overview

This short tutorial demonstrates how to run a comparative genomics workflow using a bacteria data set. Steps in the workflow combine the analyses you performed in previous submodules, and include:
- Read QC and trimming (fastqc, fastp)
- Genome assembly (SPAdes)
- Contiguity assessment (QUAST)
- Completeness assessment (BUSCO)
- Coverage assessment (BWA & Samtools)
- Taxonomic assignment (BLAST)
- Combine datasets (Blobtools)
- Genome annotation (Bakta)

The tutorial uses a popular workflow manager called [Nextflow](https://www.nextflow.io) run via [AWS Batch](https://aws.amazon.com/batch/) to automate the processes run in Submodules 1 & 2.

AWS Batch will create the needed permissions, roles and resources to run Nextflow in a serverless manner. You can set up AWS Batch manually or deploy it **automatically** with a stack template. The Launch Stack button below will take you to the cloud formation create stack webpage with the template with required resources already linked. 

If you prefer to skip manual deployment and deploy automatically in the cloud, click the Launch Stack button below. For a walkthrough of the screens during automatic deployment please click [here](https://github.com/NIGMS/NIGMS-Sandbox/blob/main/docs/HowToLaunchAWSBatch.md). The deployment should take ~5 min and then the resources will be ready for use. 

[![Launch Stack](images/LaunchStack.jpg)](https://console.aws.amazon.com/cloudformation/home?#/stacks/new?stackName=aws-batch-nigms&templateURL=https://nigms-sandbox.s3.us-east-1.amazonaws.com/cf-templates/AWSBatch_template.yaml )

Before begining this tutorial, if you do not have required roles, policies, permissions or compute environment and would like to **manually** set those up please click [here](https://github.com/NIGMS/NIGMS-Sandbox/blob/main/docs/AWS-Batch-Setup.md) to set that up.


## Prerequisites
#### Python requirements
+ Python >= 3.8

#### AWS requirements
+ Please ensure you have a VPC, subnets, and security group set up before running this tutorial.
+ Role with AdministratorAccess, AmazonSageMakerFullAccess, S3 access and AWSBatchServiceRole.
+ Instance Role with AmazonECS_FullAccess, AmazonEC2ContainerRegistryFullAccess, and S3 access.
+ If you do not have the required set-up for AWS Batch please follow this tutorial [here](https://github.com/STRIDES/NIHCloudLabAWS/blob/zbyosufzai-awsbatch-1/notebooks/AWSBatch/Intro_AWS_Batch.ipynb#install_nextflow). ***When making the instance role, make another for SageMaker notebooks with the following permissions: AdminstratorAccess, AmazonEC2ContainerRegistryFullAccess, AmazonECS_FullAccess, AmazonS3FullAccess, AmazonSageMakerFullAccess, and AWSBatchServiceRole.***
It is recommended that specific permission to folders are added through inline policy. An example of the JSON is below:

<pre>
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowSageMakerS3Access",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket",
                "s3:GetBucketLocation",
                "s3:CreateBucket"
            ],
            "Resource": [
                "arn:aws:s3:::batch-bucket",
                "arn:aws:s3:::batch-bucket/*",
                "arn:aws:s3:::nigms-sandbox-healthomics",
                "arn:aws:s3:::nigms-sandbox-healthomics/*",
                "arn:aws:s3:::ngi-igenomes",
                "arn:aws:s3:::ngi-igenomes/*"
            ]
        }
    ]
}
</pre>
For AWS bucket naming conventions, please click [here](https://docs.aws.amazon.com/AmazonS3/latest/userguide/bucketnamingrules.html).

### Step 1. Install required dependencies, update paths and create a new S3 Bucket to store input and output files (if needed)


In [ ]:
import boto3
# Get account ID and region 
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

In [ ]:
# Set variable names 
# These variables should come from the Intro AWS Batch tutorial (or leave as-is if using the launch stack button)
BUCKET_NAME = "aws-batch-nigms-batch-bucket-" + account_id
INPUT_FOLDER = 'nigms-sandbox/unh-wgsbac-pipeline'
AWS_QUEUE = 'aws-batch-nigms-JobQueue'
AWS_REGION = region

In [ ]:
# Install Nextflow
! mamba install -y -c conda-forge -c bioconda nextflow --quiet

In [ ]:
##### Import relevant libraries
# Created using this https://github.com/STRIDES/NIHCloudLabAWS/blob/zbyosufzai-awsbatch-1/notebooks/AWSBatch/Intro_AWS_Batch.ipynb#install_nextflow
#Run if you don't have Java installed
! sudo apt update
! sudo apt-get install default-jdk -y
! java -version

In [ ]:
#Install nexflow, make it exceutable, and update it
! curl https://get.nextflow.io | bash
! chmod +x nextflow
! ./nextflow self-update
! ./nextflow plugin update

In [ ]:
!pwd

In [ ]:
# replace batch bucket name in nextflow configuration file
! sed -i "s/aws-batch-nigms-batch-bucket-/$BUCKET_NAME/g" wgsbac/nextflow.config

In [ ]:
# replace job queue name in configuration file 
! sed -i "s/aws-batch-nigms-JobQueue/$AWS_QUEUE/g" wgsbac/nextflow.config

In [ ]:
# replace the region placeholder with your region
! sed -i "s/aws-region/$AWS_REGION/g" wgsbac/nextflow.config

### Step 2. Enable AWS Batch for the nextflow script 

In [ ]:
# Run nextflow script with parameters 
! ./nextflow run wgsbac/main.nf --input s3://$INPUT_FOLDER/samplesheet_test.csv -profile docker,awsbatch -c wgsbac/nextflow.config --awsqueue $AWS_QUEUE --awsregion $AWS_REGION

### Step 3: Explore Results

In [ ]:
# View output files that were output to S3 bucket
! aws s3 ls s3://$BUCKET_NAME/nextflow_output/final_reports/final_results/ --recursive | cut -c32-

In [ ]:
# Copy output to local results folder (same outdir as if workflow was run locally)
! aws s3 sync s3://$BUCKET_NAME/nextflow_output/final_reports/ wgsbac/assets/results/

In [ ]:
%%bash

# view contents of local output directory
ls wgsbac/assets/results/final_results/*

In [ ]:
%%bash

# view annotated proteome file(s) output by bakta
ls wgsbac/assets/results/final_results/bakta_results/proteomes

In [ ]:
# view blobtools plots
from IPython.display import Image
Image('wgsbac/assets/results/final_results/blobtools_plots/SRR10056829_T1_blobplot.png', width=1000)

In [ ]:
from IPython.display import Image
Image('wgsbac/assets/results/final_results/blobtools_plots/SRR10056829_T1_blobplot_read_cov.png', width=1200)